In [1]:
CONFIG_PATH = "../config.yaml"
from p_drought_indices.analysis.DeepLearning.pipeline_gwnet import data_preparation
dataloader, x_unstack, y_unstack = data_preparation(CONFIG_PATH)

The ERA5 raster has spatial dimensions: (0.10000000419197502, -0.1000000004915847)
NDVI dataset resolution: (0.043782177927351104, 0.04378217792735114)
Precipitation dataset resolution (0.10000000419197502, -0.1000000004915847)
The features have dimensions: (730, 2531, 1)
The instance have dimensions: (730, 2531, 1)
x shape:  (707, 12, 2531, 1) , y shape:  (707, 12, 2531, 1)
train x:  (511, 12, 2531, 1) y: (511, 12, 2531, 1)
val x:  (73, 12, 2531, 1) y: (73, 12, 2531, 1)
test x:  (146, 12, 2531, 1) y: (146, 12, 2531, 1)


In [8]:
dataloader["x_train"].shape

(511, 12, 2531, 1)